In [0]:
%run "../commons/configuration"

In [0]:
%run "../commons/common_functions"

In [0]:
dbutils.widgets.text("env_name","DEV")
p_env_name = dbutils.widgets.get("env_name")

In [0]:
dbutils.widgets.text("file_date","2021-03-21")
p_file_date = dbutils.widgets.get("file_date")

In [0]:
from pyspark.sql.functions import current_timestamp
from pyspark.sql.functions import lit

circuits_df = spark.read.csv(f"{raw_mount_path}/{p_file_date}/circuits.csv", 
                             header=True, 
                             inferSchema=True)\
                                .withColumnRenamed("circuitId", "circuit_id")\
                                .withColumnRenamed("circuitRef", "circuit_ref")\
                                .withColumnRenamed("lat", "latitude")\
                                .withColumnRenamed("lng", "longtitude")\
                                .withColumnRenamed("alt", "altitude")\
                                .drop("url")\
                                .withColumn("data_source",lit(p_env_name))\
                                .withColumn("file_date",lit(p_file_date))

In [0]:
circuits_df_v2 = add_ingestion_date(circuits_df)

In [0]:
#circuits_df_v2.write.parquet(f"{processed_mount_path}/circuits", mode="overwrite")
circuits_df_v2.write.mode("overwrite").format("parquet").saveAsTable("processed.circuits")

In [0]:
dbutils.notebook.exit("Success")

In [0]:
%sql
select * From processed.circuits